In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from random import shuffle

In [2]:
def get_proxies(link):  
    print('start for 1')
    response = requests.get(link)
    soup = BeautifulSoup(response.text,"lxml")
    print(response)
    print("1")
    https_proxies = filter(lambda item: "yes" in item.text,
                           soup.select("table.table tr"))
    for item in https_proxies:
        yield "{}:{}".format(item.select_one("td").text,
                             item.select_one("td:nth-of-type(2)").text)
    print('stop for 1')

In [3]:
def get_random_proxies_iter():
    print('start for 3')
    proxies = list(get_proxies('https://www.sslproxies.org/'))
    shuffle(proxies)
    print('stop for 3')
    return iter(proxies)  # iter so we can call next on it to get the next proxy


In [4]:
def get_proxy(session, proxies, validated=False):
    session.proxies = {'https': 'https://{}'.format(next(proxies))}
    if validated:
        while True:
            try:
                return session.get('https://httpbin.org/ip').json()
            except Exception:
                session.proxies = {'https': 'https://{}'.format(next(proxies))}


In [5]:
def get_response(url):
    print('start for 4')
    session = requests.Session()
    ua = UserAgent()
    proxies = get_random_proxies_iter()
    while True:
        try:
            session.headers = {'User-Agent': ua.random}
            print(get_proxy(session, proxies, validated=True))  #collect a working proxy to be used to fetch a valid response
            print('stop for 4')
            return session.get(url) # as soon as it fetches a valid response, it will break out of the while loop
        except StopIteration:
            raise  # No more proxies left to try
        except Exception:
            pass  # Other errors: try again

In [6]:
def parse_content(url):
    #uClient = uReq(url)
    #page_html = uClient.read()
    print('start for 2')
    response = get_response(url)
    soup = BeautifulSoup(response.content, 'lxml')
    containers = soup.findAll("div", {"class":"hotels-review-list-parts-SingleReview__mainCol--2XgHm"})
    print('stop for 2')
    print('m')
    #title = soup.findAll("a", {"class": "hotels-review-list-parts-ReviewTitle__reviewTitleText--3QrTy"})
    #review = soup.findAll("q", {"class": "hotels-review-list-parts-ExpandableReview__reviewText--3oMkH"})
    #ratingss = soup.find_all("div", {"class": "hotels-review-list-parts-RatingLine__bubbles--1oCI4"})
    #rat=ratingss.span
    #print(ratingss)
    #rat = soup.find_all("div", {"class": "hotels-review-list-parts-RatingLine__bubbles--1oCI4"})
    #print(rat)
    
    for container in containers:
        #rating_container = container.find_all('span.ui_bubble_rating')['class']
        #rating = rating_container[1].split('_')
        #print(rating_container)
        #rating = rating_container[1].split('_')
        #rating = rating[1]
        ratingss = container.find_all("div", {"class": "hotels-review-list-parts-RatingLine__bubbles--1oCI4"})
        for span in ratingss:
            links = span.find_all('span')
            for link in links:
                rating=link['class'][1]
        #rating_con=container.select("span[class^="ui_bubble_rating bubble_"]")
        #print(rating_con.text)
        #rating = ratingss.span.attrs['class']
        #print(ratingss)
        #rat=ratingss[0]
        #print(rating)
        title_container = container.findAll("a",{"class": "hotels-review-list-parts-ReviewTitle__reviewTitleText--3QrTy"})    
        title = title_container[0].span.span.text
        title = title.replace(","," ")
        items = title_container[0].get('href')
        if items:
            broth=BeautifulSoup(requests.get("https://www.tripadvisor.com.my"+items).text,"html.parser")
            print("https://www.tripadvisor.com.my"+items)
            review_container2 = broth.find_all("div", {"class": "prw_rup prw_reviews_resp_sur_review_text"})
            #print(review_container2)
            for item in review_container2:
                #review_container8 = item.select("div", {"class": "prw_rup prw_reviews_resp_sur_review_text"})
                #review_container8 = review_container8.get_text()
                #print(review_container8)
                review_container3 = item.find_all("span", {"class": "fullText"})
                review_container3 = review_container3[0].get_text()
                #print(len(review_container3))
                review_container3 = review_container3.replace(","," ")
                #print(review_container3)
        #review_container = container.findAll("q",{"class": "hotels-review-list-parts-ExpandableReview__reviewText--3oMkH"})
        #review = review_container[0].span.get_text()
        #review = review.replace(","," ")
        print("rating:" + rating)
        print("title:" + title)
        print("review:" + review_container3)
        f.write(rating + ", " + title + ", " + review_container3 +"\n")

In [7]:
if __name__ == '__main__':
    filename="tripAdvisor_grand129.csv"
    f=open(filename,"w",encoding='utf-8')
    headers="rating,title,review\n"
    f.write(headers)
    ind=2570
    for page in range (0,4):
        ind=ind+5
        print(ind)
        print(page)
        #url = 'https://www.hotels.com/ho223488-tr/?q-check-in=2019-08-22&q-check-out=2019-08-23&q-rooms=1&q-room-0-adults=2&SYE=3&ZSX=0&MGT=1&YGF=14&WOD=4&WOE=5&reviewTab=brand-reviews&applyEmbargo=false'
        url="https://www.tripadvisor.com.my/Hotel_Review-g298570-d3436966-Reviews-or" + str(ind) + "-Grand_Hyatt_Kuala_Lumpur-Kuala_Lumpur_Wilayah_Persekutuan.html#REVIEWS"
        print(url)
        parse_content(url) 
    f.close()

2575
0
https://www.tripadvisor.com.my/Hotel_Review-g298570-d3436966-Reviews-or2575-Grand_Hyatt_Kuala_Lumpur-Kuala_Lumpur_Wilayah_Persekutuan.html#REVIEWS
start for 2
start for 4
start for 3
start for 1
<Response [200]>
1
stop for 1
stop for 3
{'origin': '41.217.219.53, 41.217.219.53'}
stop for 4
{'origin': '200.53.45.138, 200.53.45.138'}
stop for 4
stop for 2
m
https://www.tripadvisor.com.my/ShowUserReviews-g298570-d3436966-r282282960-Grand_Hyatt_Kuala_Lumpur-Kuala_Lumpur_Wilayah_Persekutuan.html
rating:bubble_50
title:Great service and location..
review:We stayed for 4 nights and enjoy every minute of it - from the the great views from our room and the sky lobby  to the clean rooms  attentive staff and awesome pool. Location is very convenient and is close to most places of interest in KL  as well as plenty of restaurants and shopping areas...
https://www.tripadvisor.com.my/ShowUserReviews-g298570-d3436966-r283054070-Grand_Hyatt_Kuala_Lumpur-Kuala_Lumpur_Wilayah_Persekutuan.html
ratin

<Response [200]>
1
stop for 1
stop for 3
{'origin': '212.95.180.50, 212.95.180.50'}
stop for 4
stop for 2
m
https://www.tripadvisor.com.my/ShowUserReviews-g298570-d3436966-r281053759-Grand_Hyatt_Kuala_Lumpur-Kuala_Lumpur_Wilayah_Persekutuan.html
rating:bubble_50
title:Best hotel
review:Hyatt was best choice to me   good location and service. Pool and spa has nice view and service.  And I enjoyed food and drinks.  It is near KLCC and Pavillion that you will want to look arround and shopping.  I've stayed like 5days at Hyatt. and I've enjoyed most service they have like pool spa dinning room service.  for a tip   theres not many convinent store near by at late night  there are only one 7 eleven 8min walk distance. you better buy something before midnight.  Pavillion has big glossery store but they close earlier.
https://www.tripadvisor.com.my/ShowUserReviews-g298570-d3436966-r280872901-Grand_Hyatt_Kuala_Lumpur-Kuala_Lumpur_Wilayah_Persekutuan.html
rating:bubble_50
title:Great Customer Se

https://www.tripadvisor.com.my/Hotel_Review-g298570-d3672924-Reviews-The_Majestic_Hotel_Kuala_Lumpur-Kuala_Lumpur_Wilayah_Persekutuan.html